In [ ]:
# -*- coding: utf-8 -*-
"""Fork of data-loading-and-submission-preperation

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/#fileId=https%3A//storage.googleapis.com/kaggle-colab-exported-notebooks/apandeyucsd/fork-of-data-loading-and-submission-preperation.573cedf1-2111-4f0a-aeee-baa6ec54f856.ipynb%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com/20250512/auto/storage/goog4_request%26X-Goog-Date%3D20250512T102943Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Db2ad19c7b47ecf1cdb82b16924e28c01881e527f6d7df8c53c770e3e8db98dcc63af1e2c4b80d05f82b4e797e1ef3db6387be26a5bc3284fa7795f0687f3a1c39f4b430fe4ca6be561ed04c346a8f2c6c228e06f41dde3fb7677fe7b98d344b8fdbe91d9f35982ccdacf091e92f0f9714dae3d8b8d9b21a2772ed12c573c709272200aa5c2e30cff154a41aec31bb5b5fb223c5b6dcd768b5035b07026ab350f5a4afb5b1aa5af82aa7ac510bffe64f7a3ec0451fd07c39cfc0e6db09aab9c68b55973e2a25de96c6398e35bf9b65258f343749a14b24c025071a962d0b27753d0dc6a7bec45d5fd3e3d94c4e5efcf59364f757d037c70820a7a9e3ae086f368
"""

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

apandeyucsd_cse251b_path = kagglehub.dataset_download('apandeyucsd/cse251b')
print('Data source import complete.')

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from torch.nn.utils import clip_grad_norm_
from torch.optim.lr_scheduler import ReduceLROnPlateau

train_file = np.load('/kaggle/input/cse251b/train.npz')
train_data = train_file['data']
print("train_data's shape", train_data.shape)

test_file = np.load('/kaggle/input/cse251b/test_input.npz')
test_data = test_file['data']
print("test_data's shape", test_data.shape)

In [ ]:
import matplotlib.pyplot as plt

data_matrix = train_data[0]

for i in range(data_matrix.shape[0]):
    xs = data_matrix[i, :, 0]
    ys = data_matrix[i, :, 1]
    # trim all zeros
    xs = xs[xs != 0]
    ys = ys[ys != 0]
    # plot each line going from transparent to full
    plt.plot(xs, ys)

plt.show()

# 1. Prepare data: normalize and compute displacements
ego_pos = train_data[:,0,:,:2].astype(np.float32)  # (N,110,2)
# compute mean/std on input portion
mean = ego_pos[:,:50].reshape(-1,2).mean(0)
std  = ego_pos[:,:50].reshape(-1,2).std(0) + 1e-6

print((mean, std))

In [ ]:
# normalize
ego_norm = (ego_pos - mean) / std

# 1. load raw ego‐positions (N,110,2) as np.float32
ego_np = train_data[:, 0, :, :2].astype(np.float32)

# 2. compute normalization stats on the first 50 steps
obs_np = ego_np[:, :50]    # (N,50,2)
mean = obs_np.reshape(-1,2).mean(0)
std  = obs_np.reshape(-1,2).std(0) + 1e-6

# 3. normalize the entire sequence
ego_norm = (ego_np - mean) / std   # still a NumPy array

# 4. split normalized into obs & future (absolute positions)
obs_norm    = ego_norm[:, :50]     # (N,50,2)
future_norm = ego_norm[:, 50:]     # (N,60,2)

# 5. convert to torch tensors
obs_t  = torch.from_numpy(obs_norm)    # (N,50,2), dtype=torch.float32
fut_t  = torch.from_numpy(future_norm) # (N,60,2)

# 6. build per‑step displacements as a tensor
#    δ⁰ = future[:,0] – last_obs
last_obs = obs_t[:, -1:].clone()            # (N,1,2)
d0       = fut_t[:, :1] - last_obs          # (N,1,2)
#    δ¹ = future[:,1:] – future[:,:-1]
d_rest   = fut_t[:, 1:] - fut_t[:, :-1]     # (N,59,2)
#    concatenate → (N,60,2)
train_y  = torch.cat([d0, d_rest], dim=1)


train_x = obs_t

print(train_x.shape, train_y.shape)


In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=120):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        pos = torch.arange(max_len).unsqueeze(1).float()
        div = torch.exp(torch.arange(0, d_model, 2).float() * -(np.log(10000.0)/d_model))
        pe[:,0::2], pe[:,1::2] = torch.sin(pos*div), torch.cos(pos*div)
        self.pe = pe.unsqueeze(0)
    def forward(self, x): return x + self.pe[:,:x.size(1)].to(x.device)

class TrajTransformer(nn.Module):
    def __init__(self, d_model=32, nhead=2, enc_layers=2, dec_layers=2, dim_ff=64, dropout=0.1):
        super().__init__()
        self.input_lin  = nn.Linear(2, d_model)
        self.pos_enc    = PositionalEncoding(d_model)
        self.transformer= nn.Transformer(
            d_model=d_model, nhead=nhead,
            num_encoder_layers=enc_layers,
            num_decoder_layers=dec_layers,
            dim_feedforward=dim_ff, dropout=dropout,
            batch_first=True
        )
        self.out_lin    = nn.Linear(d_model, 2)
    def forward(self, src, tgt, tgt_mask):
        # src: (B,50,2) absolute positions
        # tgt: (B,59,2) previous true displacements
        src = self.pos_enc(self.input_lin(src))
        tgt = self.pos_enc(self.input_lin(tgt))
        out = self.transformer(src, tgt, tgt_mask=tgt_mask)
        return self.out_lin(out)

tgt_len = train_y.shape[1]
tgt_mask = torch.triu(torch.ones(tgt_len, tgt_len)*float('-inf'),1)

In [ ]:
def train(model, x, y, epochs=20, bs=16, lr=1e-4, device='cpu'):
    model.to(device)
    opt   = torch.optim.Adam(model.parameters(), lr=lr)
    sched = ReduceLROnPlateau(opt, 'min', patience=2, factor=0.5)
    loss_fn = nn.MSELoss()

    # — if x,y are numpy, convert; if already tensor, use as‑is
    if isinstance(x, np.ndarray):
        x = torch.from_numpy(x)
    if isinstance(y, np.ndarray):
        y = torch.from_numpy(y)

    # now x,y are Tensors
    ds = TensorDataset(x, y)
    loader = DataLoader(ds, batch_size=bs, shuffle=True, num_workers=0)

    for epoch in range(1, epochs+1):
        model.train()
        total = 0.0
        for src, disp in loader:
            src, disp = src.to(device), disp.to(device)
            dec_in = torch.cat([torch.zeros(src.size(0),1,2,device=device),
                                disp[:,:-1]], dim=1)
            pred   = model(src, dec_in, tgt_mask.to(device))
            loss   = loss_fn(pred, disp)
            opt.zero_grad()
            loss.backward()
            clip_grad_norm_(model.parameters(), 1.0)
            opt.step()
            total += loss.item()
        avg = total/len(loader)
        sched.step(avg)
        print(f"Epoch {epoch:02d} — MSE: {avg:.4f}")
    return model

In [ ]:
# initialize & train
model = TrajTransformer()
model = train(model, train_x, train_y, epochs=15, bs=16, lr=1e-4)

In [ ]:
def predict(model, test_raw):
    model.eval()
    # normalize
    tnorm = (test_raw - mean) / std          # (2100,50,2)
    src   = torch.from_numpy(tnorm).float()  # we'll take [:,:50] implicitly
    preds = []
    with torch.no_grad():
        for i in range(src.size(0)):
            s      = src[i:i+1]              # (1,50,2)
            dec_in = torch.zeros(1,60,2)     # placeholders
            # generate one delta at a time (or feed zeros if you did full teacher-forcing)
            out = model(s, dec_in, tgt_mask.to(s.device))   # (1,60,2)
            # cumulative sum → normalized future positions
            pref = tnorm[i,-1]               # (2,)
            fut  = torch.cumsum(out, dim=1) + pref  # (1,60,2)
            # de-normalize
            abs_ = fut * std + mean          # (1,60,2)
            preds.append(abs_[0])
    return torch.stack(preds).cpu().numpy()  # (2100,60,2)

In [ ]:
pred = predict(model, test_data[:,0,:,:2])
df   = pd.DataFrame(pred.reshape(-1,2), columns=['x','y'])
df.index.name = 'index'
df.to_csv('transformer_submission.csv')

In [ ]:
from IPython.display import HTML
def create_download_link(title = "Download CSV file", filename = "data.csv"):
    html = '<a href={filename}>{title}</a>'
    html = html.format(title=title,filename=filename)
    return HTML(html)

# create a link to download the dataframe which was saved with .to_csv method
create_download_link(filename='transformer_submission.csv')